___

<a href='https://www.udemy.com/user/joseportilla/'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../DATA/penguins_size.csv")

In [ ]:
df = df.dropna()
df.head()

In [ ]:
X = pd.get_dummies(df.drop('species', axis=1), drop_first=True)
y = df['species']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10, max_features='auto', random_state=101)
rfc.fit(X_train, y_train)

In [ ]:
preds = rfc.predict(X_test)
preds

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix

In [ ]:
plot_confusion_matrix(rfc, X_test, y_test)

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
rfc.feature_importances_

# Random Forest - Classification

## The Data

We will be using the same dataset through our discussions on classification with tree-methods (Decision Tree,Random Forests, and Gradient Boosted Trees) in order to compare performance metrics across these related models.

We will work with the "Palmer Penguins" dataset, as it is simple enough to help us fully understand how changing hyperparameters can change classification results.


<img src="penguin.jpg" style="max-width:400px">

Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network.

Gorman KB, Williams TD, Fraser WR (2014) Ecological Sexual Dimorphism and Environmental Variability within a Community of Antarctic Penguins (Genus Pygoscelis). PLoS ONE 9(3): e90081. doi:10.1371/journal.pone.0090081

Summary:
The data folder contains two CSV files. For intro courses/examples, you probably want to use the first one (penguins_size.csv).

* penguins_size.csv: Simplified data from original penguin data sets. Contains variables:

    * species: penguin species (Chinstrap, Adélie, or Gentoo)
    * culmen_length_mm: culmen length (mm)
    * culmen_depth_mm: culmen depth (mm)
    * flipper_length_mm: flipper length (mm)
    * body_mass_g: body mass (g)
    * island: island name (Dream, Torgersen, or Biscoe) in the Palmer Archipelago (Antarctica)
    * sex: penguin sex

* (Not used) penguins_lter.csv: Original combined data for 3 penguin species  

Note: The culmen is "the upper ridge of a bird's beak" 

**Our goal is to create a model that can help predict a species of a penguin based on physical attributes, then we can use that model to help researchers classify penguins in the field, instead of needing an experienced biologist**

## Imports

In [ ]:
df = df.dropna()
df.head()

## Train | Test Split

In [ ]:
X = pd.get_dummies(df.drop('species',axis=1),drop_first=True)
y = df['species']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Random Forest Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
help(RandomForestClassifier)

In [ ]:
# Use 10 random trees
model = RandomForestClassifier(n_estimators=10,max_features='auto',random_state=101)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
preds = model.predict(X_test)

## Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix,accuracy_score

In [ ]:
confusion_matrix(y_test,preds)

In [ ]:
plot_confusion_matrix(model,X_test,y_test)

## Feature Importance

Very useful attribute of the trained model!

In [ ]:
model.feature_importances_

## Choosing correct number of trees

Let's explore if continually adding more trees improves performance...

In [ ]:
test_error = []

for n in range(1,40):
    # Use n random trees
    model = RandomForestClassifier(n_estimators=n,max_features='auto')
    model.fit(X_train,y_train)
    test_preds = model.predict(X_test)
    test_error.append(1-accuracy_score(test_preds,y_test))
 

In [ ]:
plt.plot(range(1,40),test_error,label='Test Error')
plt.legend()

Clearly there are diminishing returns, on such a small dataset, we've pretty much extracted all the information we can after about 5 trees.

# Random Forest - HyperParameter Exploration

https://archive.ics.uci.edu/ml/datasets/banknote+authentication

In [ ]:
df = pd.read_csv("../DATA/data_banknote_authentication.csv")
df.head()

In [ ]:
sns.pairplot(data=df, hue='Class')

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble        import RandomForestClassifier

In [ ]:
n_estimators = [64, 100, 128, 200]
max_features = [2, 3, 4]
bootstrap    = [True, False]
oob_score    = [True, False]

param_grid   = {'n_estimators': n_estimators,
                'max_features': max_features,
                'bootstrap': bootstrap,
                'oob_score': oob_score}

rfc = RandomForestClassifier()

grid = GridSearchCV(estimator=rfc, param_grid=param_grid)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
rfc = RandomForestClassifier(max_features=2, n_estimators=200, oob_score=True)
rfc.fit(X_train, y_train)

In [ ]:
rfc.oob_score_

In [ ]:
predictions = rfc.predict(X_test)

from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score
print(classification_report(y_test, predictions))

In [ ]:
plot_confusion_matrix(rfc, X_test, y_test)

In [ ]:
errors = []
misclassifications = []

for n in range(1, 200):
    rfc = RandomForestClassifier(n_estimators=n, max_features=2)
    rfc.fit(X_train, y_train)
    preds = rfc.predict(X_test)
    err = 1 - accuracy_score(y_test, preds)
    n_missed = np.sum(preds != y_test)
    
    errors.append(err)
    misclassifications.append(n_missed)

In [ ]:
plt.figure(dpi=120)
plt.plot(range(1, 200), errors, c='r')

In [ ]:
plt.figure(dpi=120)
plt.plot(range(1, 200), misclassifications)

In [ ]:
df.head()

In [ ]:
sns.pairplot(df,hue='Class')

In [ ]:
X = df.drop("Class",axis=1)

In [ ]:
y = df["Class"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
n_estimators=[64,100,128,200]
max_features= [2,3,4]
bootstrap = [True,False]
oob_score = [True,False]

In [ ]:
param_grid = {'n_estimators':n_estimators,
             'max_features':max_features,
             'bootstrap':bootstrap,
             'oob_score':oob_score}  # Note, oob_score only makes sense when bootstrap=True!

In [ ]:
rfc = RandomForestClassifier()
grid = GridSearchCV(rfc,param_grid)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
predictions = grid.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
plot_confusion_matrix(grid,X_test,y_test)

In [ ]:
# No underscore, reports back original oob_score parameter
grid.best_estimator_.oob_score

In [ ]:
# With underscore, reports back fitted attribute of oob_score
grid.best_estimator_.oob_score_

## Understanding Number of Estimators (Trees)

Let's plot out error vs. Number of Estimators

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
errors = []
misclassifications = []

for n in range(1,64):
    rfc = RandomForestClassifier( n_estimators=n,bootstrap=True,max_features= 2)
    rfc.fit(X_train,y_train)
    preds = rfc.predict(X_test)
    err = 1 - accuracy_score(preds,y_test)
    n_missed = np.sum(preds != y_test) # watch the video to understand this line!!
    errors.append(err)
    misclassifications.append(n_missed)

In [ ]:
plt.plot(range(1,64),errors)

In [ ]:
plt.plot(range(1,64),misclassifications)